In [ ]:
# http://ihongss.com/webboard/editor?mode=v

In [3]:
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.master("local[*]").appName("spark_app1").getOrCreate()
# spark = SparkSession.builder.master("spark://172.30.32.173:7077").appName("spark_app1").getOrCreate()

df1 = spark.createDataFrame([(1, "andy", 20, "USA"), (2, "jeff", 23, "China"), (3, "james", 18, "USA")]).toDF("id", "name", "age", "country")
df1.printSchema
df1.show()

df1.createOrReplaceTempView("df_table")
spark.sql("SELECT * FROM df_table WHERE age < 20").show()

+---+-----+---+-------+
| id| name|age|country|
+---+-----+---+-------+
|  1| andy| 20|    USA|
|  2| jeff| 23|  China|
|  3|james| 18|    USA|
+---+-----+---+-------+

+---+-----+---+-------+
| id| name|age|country|
+---+-----+---+-------+
|  3|james| 18|    USA|
+---+-----+---+-------+



In [8]:
# ML 예제

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ml_log').master('local[*]').getOrCreate()

# 훈련용 데이터(키, 몸무게, 나이, 성별)
train = spark.createDataFrame([(160.0, 65.3, 40, 1.0), 
                               (166.0, 35.3, 20, 0.0), 
                               (180.0, 65.3, 40, 1.0)]).toDF('height', 'weight', 'age', 'gender')
# 테스트용(키, 몸무게, 나이)
test = spark.createDataFrame([(160.0, 56.5, 34)]).toDF('height', 'weight', 'age')

# vector assemble (1번 항목)
assemble = VectorAssembler(inputCols = ['height', 'weight', 'age'], outputCol='features')

# train 데이터에 features 추가
assembled_train = assemble.transform(train)
assembled_train.show()

# 모델 생성 알고리즘 (2번 항목)
lr = LogisticRegression(maxIter = 10, regParam = 0.01, labelCol = 'gender')

# 모델 생성
model = lr.fit(assembled_train)

# 예측값 확인
model.transform(assembled_train).show()

# 파이프라인
pipeline = Pipeline(stages = [assemble, lr])

# 파이프라인 모델 생성
pipelineModel = pipeline.fit(train)

# 파이프라인을 이용한 예측값
pipelineModel.transform(train).show()

# 모델 저장
path1="C:/data/spark/regression_model"
model.write().overwrite().save(path1)  # 쓰기
loadModel = LogisticRegressionModel.load(path1) #읽기 

# 파이프라인 모델 저장
path2="c:/data/spark/pipeline_model"
pipelineModel.write().overwrite().save(path2)  # 쓰기
loadpipeModel = PipelineModel.load(path2)    # 읽기

+------+------+---+------+-----------------+
|height|weight|age|gender|         features|
+------+------+---+------+-----------------+
| 160.0|  65.3| 40|   1.0|[160.0,65.3,40.0]|
| 166.0|  35.3| 20|   0.0|[166.0,35.3,20.0]|
| 180.0|  65.3| 40|   1.0|[180.0,65.3,40.0]|
+------+------+---+------+-----------------+

+------+------+---+------+-----------------+--------------------+--------------------+----------+
|height|weight|age|gender|         features|       rawPrediction|         probability|prediction|
+------+------+---+------+-----------------+--------------------+--------------------+----------+
| 160.0|  65.3| 40|   1.0|[160.0,65.3,40.0]|[-4.7251940334141...|[0.00879102532117...|       1.0|
| 166.0|  35.3| 20|   0.0|[166.0,35.3,20.0]|[3.17760255117735...|[0.95998266728207...|       0.0|
| 180.0|  65.3| 40|   1.0|[180.0,65.3,40.0]|[-3.2900693199985...|[0.03591344569278...|       1.0|
+------+------+---+------+-----------------+--------------------+--------------------+----------

Py4JJavaError: An error occurred while calling o1059.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:100)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDDFunctions$